# Linear Regression Experiment

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import numpy as np

train_file = np.load('../../../train.npz')
# train_file = np.load('/kaggle/input/cse-251-b-2025/train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)

test_file = np.load('../../../test_input.npz')
# test_file = np.load('/kaggle/input/cse-251-b-2025/test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)

train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


In [20]:
train_data = train_data[..., :6]
test_data = test_data[..., :6]
print("train_data's shape", train_data.shape)
print("test_data's shape", test_data.shape)

train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


In [21]:
import sys
sys.path.append('../..')
from TrajectoryDataset import TrajectoryDatasetTrain, TrajectoryDatasetTest
from utils import train_model

sys.path.append('../../..')
from models.linear_regression import LinearRegressionModel

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch_geometric.data import Batch

In [23]:
torch.manual_seed(251)
np.random.seed(42)

scale = 5.0

N = len(train_data)
val_size = int(0.05 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using Apple Silicon GPU


In [24]:
model = LinearRegressionModel()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25) # You can try different schedulers
early_stopping_patience = 15
criterion = nn.MSELoss()

In [25]:
model = model.to(device)
train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    device, 
    optimizer, 
    criterion, 
    scheduler, 
    early_stopping_patience,
    "linear_regression_weights"
)

Epoch:   1%|          | 1/100 [00:04<06:42,  4.06s/epoch]

Epoch 000 | Learning rate 0.000100 | train normalized MSE 9821.2405 | val normalized MSE 4369.0214, | val unnormalized MAE 212.9531 | val unnormalized MSE 109225.5371


Epoch:   2%|▏         | 2/100 [00:07<06:03,  3.71s/epoch]

Epoch 001 | Learning rate 0.000100 | train normalized MSE 3830.6212 | val normalized MSE 2866.9605, | val unnormalized MAE 173.0659 | val unnormalized MSE 71674.0132


Epoch:   3%|▎         | 3/100 [00:11<05:59,  3.71s/epoch]

...

Epoch 097 | Learning rate 0.000000 | train normalized MSE 190.6593 | val normalized MSE 306.5788, | val unnormalized MAE  55.8530 | val unnormalized MSE 7664.4697


Epoch:  99%|█████████▉| 99/100 [05:52<00:03,  3.45s/epoch]

Epoch 098 | Learning rate 0.000000 | train normalized MSE 190.1146 | val normalized MSE 306.6116, | val unnormalized MAE  55.8374 | val unnormalized MSE 7665.2890


Epoch: 100%|██████████| 100/100 [05:55<00:00,  3.55s/epoch]

Epoch 099 | Learning rate 0.000000 | train normalized MSE 190.4754 | val normalized MSE 307.0003, | val unnormalized MAE  55.8687 | val unnormalized MSE 7675.0079


In [26]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

mlp_weights = torch.load("linear_regression_weights.pt")
model.load_state_dict(mlp_weights)

# also save entire model
torch.save(model, "full_linear_regression_model.pt")

test_dataset.generate_submission_predictions(model, device, output_file_name='linear_regression_submission')